In [8]:
# checking bounding boxes

from PIL import Image, ImageDraw
import json

img = Image.open("/mnt/windows/Users/Aditya Bhandari/Desktop/code/GitHub/Inscription-character-deciphering-for-Brahmi-script/0_clean.png").convert("RGB")
draw = ImageDraw.Draw(img)

with open("/mnt/windows/Users/Aditya Bhandari/Desktop/code/GitHub/Inscription-character-deciphering-for-Brahmi-script/0.json", "r", encoding="utf-8") as f:
    data = json.load(f)

for item in data:
    x1, y1, x2, y2 = item["bbox"]
    draw.rectangle(
        [x1, y1, x2, y2],
        outline="red",
        width=2
    )

img.save("boxed.png")


In [ ]:
# full preprocessing

import cv2
import os
for i in range(1000):
    img_path = f"/mnt/windows/Users/Aditya Bhandari/Downloads/output/imgs/{i}_engraved.png"
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    denoise = cv2.medianBlur(gray, 3)
    _, thresh = cv2.threshold(
        denoise, 0, 255,
        cv2.THRESH_BINARY + cv2.THRESH_OTSU
    )
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    clean = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (4, 4))
    clean_ = cv2.morphologyEx(clean, cv2.MORPH_CLOSE, kernel)
    save_path = f"/mnt/windows/Users/Aditya Bhandari/Downloads/output/imgs_clean/{i}_clean.png"
    cv2.imwrite(save_path, clean_)

In [9]:
import json
import os
import cv2
import glob
from sklearn.model_selection import train_test_split
import shutil

# --- Configuration ---
RAW_DATA_DIR = "/mnt/windows/Users/adity/Downloads/output_clean/"
OUTPUT_DIR = "datasets/brahmi_ocr"

def convert_bbox_to_yolo(bbox, img_width, img_height):
    xmin, ymin, xmax, ymax = bbox
    
    dw = 1.0 / img_width
    dh = 1.0 / img_height
    
    w = xmax - xmin
    h = ymax - ymin
    x_center = xmin + w / 2.0
    y_center = ymin + h / 2.0
    
    x_center *= dw
    w *= dw
    y_center *= dh
    h *= dh
    
    return x_center, y_center, w, h

def main():
    # 1. Identify all valid pairs
    print(f"Scanning {RAW_DATA_DIR}...")
    valid_pairs = [] 
    
    for i in range(1000):
        json_filename = f"{i}.json"
        img_filename = f"{i}_clean.png"
        
        json_path = os.path.join(RAW_DATA_DIR, json_filename)
        img_path = os.path.join(RAW_DATA_DIR, img_filename)
        
        if os.path.exists(json_path) and os.path.exists(img_path):
            valid_pairs.append((json_path, img_path, str(i)))
    
    print(f"Found {len(valid_pairs)} valid image-JSON pairs.")

    if len(valid_pairs) == 0:
        print("Error: No data found. Check your paths.")
        return

    # 2. Build Class Vocabulary
    print("Building class vocabulary...")
    unique_chars = set()
    
    for json_path, _, _ in valid_pairs:
        with open(json_path, 'r', encoding='utf-8') as f:
            try:
                data = json.load(f)
                for item in data:
                    # FIX: Check if 'char' exists before accessing it
                    if 'char' in item:
                        unique_chars.add(item['char'])
            except json.JSONDecodeError:
                print(f"Skipping corrupted JSON: {json_path}")

    char_to_id = {char: idx for idx, char in enumerate(sorted(list(unique_chars)))}
    id_to_char = {idx: char for char, idx in char_to_id.items()}
    
    print(f"Found {len(char_to_id)} unique characters.")
    
    # Save mapping
    with open("class_mapping.json", "w", encoding='utf-8') as f:
        json.dump(char_to_id, f, ensure_ascii=False, indent=2)

    # 3. Prepare Directories
    for split in ['train', 'val']:
        os.makedirs(os.path.join(OUTPUT_DIR, 'images', split), exist_ok=True)
        os.makedirs(os.path.join(OUTPUT_DIR, 'labels', split), exist_ok=True)

    # 4. Split Data
    train_pairs, val_pairs = train_test_split(valid_pairs, test_size=0.2, random_state=42)
    splits = {'train': train_pairs, 'val': val_pairs}

    # 5. Process Files
    print("Converting and moving files...")
    for split, pairs in splits.items():
        for json_path, img_path, base_id in pairs:
            
            img = cv2.imread(img_path)
            if img is None: continue
            height, width = img.shape[:2]

            with open(json_path, 'r', encoding='utf-8') as f:
                annotations = json.load(f)

            yolo_lines = []
            for ann in annotations:
                # FIX: Skip entries without 'char'
                if 'char' not in ann:
                    continue
                
                char = ann['char']
                
                # Verify char is in our list (it should be)
                if char not in char_to_id: 
                    continue
                
                cls_id = char_to_id[char]
                
                # Check if 'bbox' exists too
                if 'bbox' not in ann:
                    continue
                    
                bbox = ann['bbox'] 
                xc, yc, w, h = convert_bbox_to_yolo(bbox, width, height)
                
                if w > 0 and h > 0:
                    yolo_lines.append(f"{cls_id} {xc} {yc} {w} {h}")

            # Only write files if we found valid lines
            if len(yolo_lines) > 0:
                label_out_path = os.path.join(OUTPUT_DIR, 'labels', split, base_id + ".txt")
                with open(label_out_path, 'w', encoding='utf-8') as f:
                    f.write("\n".join(yolo_lines))

                out_img_name = base_id + ".png"
                shutil.copy(img_path, os.path.join(OUTPUT_DIR, 'images', split, out_img_name))

    # 6. Create YAML Config
    yaml_content = f"""
path: {os.path.abspath(OUTPUT_DIR)}
train: images/train
val: images/val

nc: {len(char_to_id)}
names: {list(id_to_char.values())}
    """
    
    with open("brahmi_config.yaml", "w", encoding='utf-8') as f:
        f.write(yaml_content)
        
    print("Data preparation complete. 'brahmi_config.yaml' created.")

if __name__ == "__main__":
    main()

Scanning /mnt/windows/Users/adity/Downloads/output_clean/...
Found 1000 valid image-JSON pairs.
Building class vocabulary...
Found 0 unique characters.
Converting and moving files...
Data preparation complete. 'brahmi_config.yaml' created.


In [ ]:
# separating pngs and jsons
import os
import shutil
import glob 


src_dir = "/mnt/windows/Users/Aditya Bhandari/Downloads/output/"
imgs_dir = os.path.join(src_dir, "imgs")
jsons_dir = os.path.join(src_dir, "jsons")
os.makedirs(imgs_dir, exist_ok=True)
os.makedirs(jsons_dir, exist_ok=True)
for file_path in glob.glob(os.path.join(src_dir, "*")):
    if file_path.endswith(".png"):
        shutil.move(file_path, os.path.join(imgs_dir, os.path.basename(file_path)))
    elif file_path.endswith(".json"):
        shutil.move(file_path, os.path.join(jsons_dir, os.path.basename(file_path)))


In [ ]:
# clean to yolo format

import json
import os
from pathlib import Path
from PIL import Image

# ------------------ PATHS ------------------
IMG_DIR = Path("/mnt/windows/Users/Aditya Bhandari/Downloads/output/imgs_clean")
JSON_DIR = Path("/mnt/windows/Users/Aditya Bhandari/Downloads/output/jsons")
OUT_DIR = Path("/mnt/windows/Users/Aditya Bhandari/Downloads/output/yolo_format")

IMG_EXT = ".png"   # change to ".jpg" if needed

# ------------------ OUTPUT DIRS ------------------
(OUT_DIR / "images").mkdir(parents=True, exist_ok=True)
(OUT_DIR / "labels").mkdir(parents=True, exist_ok=True)

# ------------------ CLASS MAPPING ------------------
glyph_to_class = {}
classes = []

def get_class_id(glyph):
    if glyph not in glyph_to_class:
        glyph_to_class[glyph] = len(classes)
        classes.append(glyph)
    return glyph_to_class[glyph]

# ------------------ CONVERSION ------------------
for json_file in JSON_DIR.glob("*.json"):
    x = json_file.stem  # X
    img_path = IMG_DIR / f"{x}_clean{IMG_EXT}"

    if not img_path.exists():
        print(f"Missing image: {img_path}")
        continue

    # Copy image
    out_img_path = OUT_DIR / "images" / img_path.name
    if not out_img_path.exists():
        out_img_path.write_bytes(img_path.read_bytes())

    # Load image size
    with Image.open(img_path) as img:
        img_w, img_h = img.size

    # Load annotations
    with open(json_file, "r", encoding="utf-8") as f:
        annotations = json.load(f)

    label_lines = []

    for obj in annotations:
        glyph = obj["glyph"]
        x1, y1, x2, y2 = obj["bbox"]

        class_id = get_class_id(glyph)

        # Convert to YOLO format
        cx = (x1 + x2) / 2.0 / img_w
        cy = (y1 + y2) / 2.0 / img_h
        w = (x2 - x1) / img_w
        h = (y2 - y1) / img_h

        label_lines.append(
            f"{class_id} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}"
        )

    # Write label file
    label_path = OUT_DIR / "labels" / f"{x}_clean.txt"
    with open(label_path, "w") as f:
        f.write("\n".join(label_lines))

# ------------------ SAVE CLASSES ------------------
with open(OUT_DIR / "classes.txt", "w", encoding="utf-8") as f:
    for glyph in classes:
        f.write(glyph + "\n")

print("YOLO conversion complete.")
print(f"Total classes: {len(classes)}")


YOLO conversion complete.
Total classes: 130


In [ ]:
# making config yaml
import os
import yaml  # pip install pyyaml

# --- Configuration ---
# Path to the folder containing 'images', 'labels', and 'classes.txt'
DATASET_ROOT = os.path.abspath("data/yolo_format") 
CLASSES_FILE = os.path.join(DATASET_ROOT, "classes.txt")

def generate_yaml():
    if not os.path.exists(CLASSES_FILE):
        print(f"Error: classes.txt not found at {CLASSES_FILE}")
        return

    # 1. Read class names from classes.txt
    # We assume Line 1 = ID 0, Line 2 = ID 1, etc.
    with open(CLASSES_FILE, "r", encoding="utf-8") as f:
        class_names = [line.strip() for line in f.readlines() if line.strip()]

    print(f"Found {len(class_names)} classes.")

    # 2. Define Image Paths
    # If you haven't split into train/val folders, we will use the same 'images' folder for both.
    # (Note: For proper training, you should split them, but this gets it running).
    train_path = "images"
    val_path = "images" 
    
    # Check if user actually has 'train' and 'val' subfolders
    if os.path.exists(os.path.join(DATASET_ROOT, "images", "train")):
        train_path = "images/train"
        val_path = "images/val" # Assumes val exists if train does

    # 3. Create Dictionary for YAML
    data = {
        "path": DATASET_ROOT,  # Absolute path to root
        "train": train_path,
        "val": val_path,
        "nc": len(class_names),
        "names": class_names   # List of class names
    }

    # 4. Save to yaml
    output_path = "brahmi_config.yaml"
    with open(output_path, "w", encoding="utf-8") as f:
        yaml.dump(data, f, allow_unicode=True, sort_keys=False)

    print(f"Success! '{output_path}' generated.")
    print(f"Training path set to: {os.path.join(DATASET_ROOT, train_path)}")

if __name__ == "__main__":
    generate_yaml()

Found 130 classes.
Success! 'brahmi_config.yaml' generated.
Training path set to: /mnt/windows/Users/Aditya Bhandari/Desktop/code/GitHub/Inscription-character-deciphering-for-Brahmi-script/data/yolo_format/images


In [ ]:
# train val split
import random
from pathlib import Path
import shutil

random.seed(42)

# Use existing DATASET_ROOT variable
images_dir = Path(DATASET_ROOT) / "images"
labels_dir = Path(DATASET_ROOT) / "labels"

if not images_dir.exists() or not labels_dir.exists():
    raise FileNotFoundError(f"Missing 'images' or 'labels' under {DATASET_ROOT}")

# Create train/val subfolders
train_img_dir = images_dir / "train"
val_img_dir = images_dir / "val"
train_lbl_dir = labels_dir / "train"
val_lbl_dir = labels_dir / "val"

for d in (train_img_dir, val_img_dir, train_lbl_dir, val_lbl_dir):
    d.mkdir(parents=True, exist_ok=True)

# Collect image files at root of images/ (exclude anything already in train/ or val/)
img_files = sorted([p for p in images_dir.glob("*_clean.*") if p.is_file() and p.parent == images_dir and p.suffix.lower() in (".png", ".jpg", ".jpeg")])

pairs = []
for img in img_files:
    base = img.stem.replace("_clean", "")
    label = labels_dir / f"{base}_clean.txt"
    if label.exists():
        pairs.append((img, label))
    else:
        print(f"Warning: label missing for {img.name}")

if not pairs:
    print("No image/label pairs found to split.")
else:
    random.shuffle(pairs)
    split_idx = int(len(pairs) * 0.8)
    train_pairs = pairs[:split_idx]
    val_pairs = pairs[split_idx:]

    def move_pairs(pairs, img_dest, lbl_dest):
        for img, lbl in pairs:
            # Skip if already moved
            if img.exists():
                shutil.move(str(img), str(img_dest / img.name))
            if lbl.exists():
                shutil.move(str(lbl), str(lbl_dest / lbl.name))

    move_pairs(train_pairs, train_img_dir, train_lbl_dir)
    move_pairs(val_pairs, val_img_dir, val_lbl_dir)

    print(f"Created train/ and val/ and moved {len(train_pairs)} train, {len(val_pairs)} val pairs.")

Created train/ and val/ and moved 800 train, 200 val pairs.


In [1]:
# ocr training
from ultralytics import YOLO

def train():
    # 1. Load a pretrained model (YOLOv8n is 'nano' - fastest, good for simple shapes like chars)
    # You can use 'yolov8s.pt' (small) or 'yolov8m.pt' (medium) for higher accuracy but slower speed.
    model = YOLO("yolov8n.pt") 

    # 2. Train the model
    results = model.train(
        data="brahmi_config.yaml",  # Path to the yaml created in Step 1
        epochs=100,                 # Adjust based on dataset size (50-100 is usually good start)
        imgsz=640,                  # Image size
        batch=8,                   # Adjust based on your GPU VRAM
        device='cpu',                   # Use 0 for GPU, 'cpu' for CPU
        patience=20,
        name="brahmi_ocr_run",       # Name of the run folder
        lr0=0.003,
        workers=4
    )

    # 3. Export the model (optional) to ONNX for easier deployment later
    model.export(format="onnx")

if __name__ == "__main__":
    train()

ModuleNotFoundError: No module named 'ultralytics'

In [ ]:
# ocr inference
from ultralytics import YOLO
import json

# Load mapping to convert IDs back to Characters
with open("class_mapping.json", "r", encoding='utf-8') as f:
    mapping = json.load(f)
    # The JSON keys are chars, values are IDs. We need the reverse.
    id_to_char = {v: k for k, v in mapping.items()}

# Load your trained model
model = YOLO("runs/detect/brahmi_ocr_run/weights/best.pt")

# Run inference
results = model.predict("path/to/test_image.jpg")

# Process results
for result in results:
    boxes = result.boxes
    for box in boxes:
        cls_id = int(box.cls[0])
        char = id_to_char.get(cls_id, "?")
        coords = box.xyxy[0].tolist() # [xmin, ymin, xmax, ymax]
        
        print(f"Detected: {char} at {coords}")

    # Show the image with boxes
    result.show()

In [ ]:
# postproc

from ultralytics import YOLO
import json
import numpy as np

# --- Configuration ---
MODEL_PATH = "runs/detect/brahmi_ocr_run/weights/best.pt"
IMG_PATH = "path/to/your/test_image.png"
MAPPING_PATH = "class_mapping.json"

def load_mapping(path):
    with open(path, 'r', encoding='utf-8') as f:
        mapping = json.load(f)
    # Invert mapping: ID (int) -> Char (str)
    return {v: k for k, v in mapping.items()}

def get_vertical_center(bbox):
    # bbox = [x1, y1, x2, y2]
    return (bbox[1] + bbox[3]) / 2

def boxes_to_text(results, id_to_char):
    """
    Converts YOLO results into a structured string, handling multiple lines.
    """
    # 1. Extract raw data from YOLO results
    # Each detection is: [x1, y1, x2, y2, conf, class_id]
    boxes = results[0].boxes.data.cpu().numpy()
    
    if len(boxes) == 0:
        return ""

    # Convert to list of dicts for easier handling
    detections = []
    for box in boxes:
        x1, y1, x2, y2, conf, cls_id = box
        detections.append({
            "bbox": [x1, y1, x2, y2],
            "center_y": (y1 + y2) / 2,
            "x1": x1,
            "char": id_to_char.get(int(cls_id), "?")
        })

    # 2. Sort primarily by Top-Down (y1) to handle the initial order
    detections.sort(key=lambda k: k['bbox'][1])

    # 3. Group into lines (Line Bucketing)
    lines = []
    
    while len(detections) > 0:
        # Start a new line with the top-most remaining box
        line_boxes = [detections.pop(0)]
        
        # Define the vertical bounds of this line based on the first char
        # We use a slight tolerance (e.g., usually a line is defined by the char height)
        ref_box = line_boxes[0]['bbox']
        ref_h = ref_box[3] - ref_box[1]
        ref_center = line_boxes[0]['center_y']
        
        # Check remaining boxes to see if they belong to this line
        # Logic: If the box's center is within the vertical span of the reference box
        # (plus a small margin), it belongs to this line.
        
        # We iterate backwards so we can remove items safely
        i = 0
        while i < len(detections):
            curr = detections[i]
            # Check vertical overlap / proximity
            # If the center of the char is reasonably close to the center of the line reference
            if abs(curr['center_y'] - ref_center) < (ref_h * 0.6): # 0.6 is a tolerance factor
                line_boxes.append(detections.pop(i))
            else:
                i += 1
        
        # 4. Sort the identified line Left-to-Right
        line_boxes.sort(key=lambda k: k['x1'])
        
        # 5. Join characters to form the line string
        line_text = "".join([item['char'] for item in line_boxes])
        lines.append(line_text)

    # 6. Join all lines with newlines
    full_text = "\n".join(lines)
    return full_text

def main():
    # Load resources
    id_to_char = load_mapping(MAPPING_PATH)
    model = YOLO(MODEL_PATH)

    # Run Inference
    results = model.predict(IMG_PATH)

    # Convert to text
    final_text = boxes_to_text(results, id_to_char)
    
    print("--- Recognized Text ---")
    print(final_text)
    print("-----------------------")

if __name__ == "__main__":
    main()

In [6]:
# sanity check

import json
import cv2
import numpy as np
from pathlib import Path

# --- Configuration ---
IMG_PATH = "/mnt/windows/Users/Aditya Bhandari/Desktop/code/GitHub/Inscription-character-deciphering-for-Brahmi-script/data/yolo_format/images/train/0_clean.png"
LABEL_PATH = "/mnt/windows/Users/Aditya Bhandari/Desktop/code/GitHub/Inscription-character-deciphering-for-Brahmi-script/data/yolo_format/labels/train/0_clean.txt"   # YOLO label file
CLASSES_PATH = "/mnt/windows/Users/Aditya Bhandari/Desktop/code/GitHub/Inscription-character-deciphering-for-Brahmi-script/data/yolo_format/classes.txt"

# --------------------

def load_classes_txt(path):
    """
    Reads classes.txt where each line corresponds to a class ID.
    Line number = class ID.
    """
    with open(path, "r", encoding="utf-8") as f:
        lines = [line.strip() for line in f if line.strip()]

    return {i: char for i, char in enumerate(lines)}

def yolo_to_xyxy(label, img_w, img_h):
    """
    Converts YOLO normalized bbox to pixel xyxy.
    """
    cls, xc, yc, w, h = label
    xc *= img_w
    yc *= img_h
    w *= img_w
    h *= img_h

    x1 = xc - w / 2
    y1 = yc - h / 2
    x2 = xc + w / 2
    y2 = yc + h / 2

    return int(cls), x1, y1, x2, y2

def boxes_to_text_from_labels(label_path, img_shape, id_to_char):
    img_h, img_w = img_shape[:2]

    # Load labels
    labels = np.loadtxt(label_path).reshape(-1, 5)

    detections = []
    for lab in labels:
        cls, x1, y1, x2, y2 = yolo_to_xyxy(lab, img_w, img_h)
        detections.append({
            "bbox": [x1, y1, x2, y2],
            "center_y": (y1 + y2) / 2,
            "x1": x1,
            "char": id_to_char.get(cls, "?")
        })

    if not detections:
        return ""

    # Sort top → bottom
    detections.sort(key=lambda d: d["bbox"][1])

    lines = []

    while detections:
        line = [detections.pop(0)]
        ref = line[0]
        ref_h = ref["bbox"][3] - ref["bbox"][1]
        ref_center = ref["center_y"]

        i = 0
        while i < len(detections):
            curr = detections[i]
            if abs(curr["center_y"] - ref_center) < ref_h * 0.6:
                line.append(detections.pop(i))
            else:
                i += 1

        # left → right
        line.sort(key=lambda d: d["x1"])
        lines.append("".join(d["char"] for d in line))

    return "\n".join(lines)

def main():
    id_to_char = load_classes_txt(CLASSES_PATH)

    img = cv2.imread(IMG_PATH)
    assert img is not None, "Image not found"

    text = boxes_to_text_from_labels(
        LABEL_PATH,
        img.shape,
        id_to_char
    )

    print("---- Ground Truth Text ----")
    print(text)
    print("---------------------------")


if __name__ == "__main__":
    main()


---- Ground Truth Text ----
𑀫𑀗𑁆𑀓𑁃𑀬𑀽𑀭𑁆𑀦𑁂𑀝𑀼𑀫𑀸𑀭𑀦𑁆𑀘𑁂𑀬𑁆𑀢𑀢𑀸𑀦𑀫𑁆𑁇𑀧𑀼𑀓𑀴𑀽𑀭𑁆
𑀘𑀫𑀡𑀫𑀼𑀦𑀺𑀯𑀭𑁆𑀓𑁆𑀓𑀼𑀇𑀧𑁆𑀧𑀴𑁆𑀴𑀺𑀅𑀫𑀡𑀧𑁆𑀧𑀸𑀴𑀺𑁇𑀅𑀭𑀫𑁆
𑀯𑁂𑀮𑁆𑀓𑁇
---------------------------


In [ ]:
# spacing code

from ultralytics import YOLO
import json
import statistics

# --- Configuration ---
MODEL_PATH = "runs/detect/brahmi_ocr_run/weights/best.pt"
IMG_PATH = "path/to/test_image.png"
MAPPING_PATH = "class_mapping.json"
SPACE_THRESHOLD_RATIO = 0.5  # If gap > 50% of median char width, it's a space

def load_mapping(path):
    with open(path, 'r', encoding='utf-8') as f:
        mapping = json.load(f)
    return {v: k for k, v in mapping.items()}

def boxes_to_text_with_spaces(results, id_to_char):
    """
    Converts YOLO results into structured text with spaces for words.
    """
    # 1. Extract raw data
    boxes = results[0].boxes.data.cpu().numpy()
    if len(boxes) == 0: return ""

    detections = []
    for box in boxes:
        x1, y1, x2, y2, conf, cls_id = box
        detections.append({
            "bbox": [x1, y1, x2, y2],
            "center_y": (y1 + y2) / 2,
            "x1": x1,
            "x2": x2,
            "width": x2 - x1,
            "char": id_to_char.get(int(cls_id), "?")
        })

    # 2. Sort Top-Down to handle lines
    detections.sort(key=lambda k: k['bbox'][1])

    # 3. Line Bucketing (Same as before)
    lines_of_words = []
    
    while len(detections) > 0:
        line_boxes = [detections.pop(0)]
        ref_h = line_boxes[0]['bbox'][3] - line_boxes[0]['bbox'][1]
        ref_center = line_boxes[0]['center_y']
        
        i = 0
        while i < len(detections):
            curr = detections[i]
            if abs(curr['center_y'] - ref_center) < (ref_h * 0.6):
                line_boxes.append(detections.pop(i))
            else:
                i += 1
        
        # 4. Sort Left-to-Right
        line_boxes.sort(key=lambda k: k['x1'])
        
        # --- NEW: Word Segmentation Logic ---
        
        # Calculate median width for this specific line (adaptive)
        widths = [b['width'] for b in line_boxes]
        median_width = statistics.median(widths) if widths else 0
        
        # Define the pixel distance that counts as a space
        # You might need to tweak the 0.5 based on your specific font spacing
        threshold = median_width * SPACE_THRESHOLD_RATIO 
        
        line_str = ""
        words_list = [] # If you want a list of words specifically
        current_word = ""

        for j in range(len(line_boxes)):
            char = line_boxes[j]['char']
            current_word += char
            line_str += char
            
            # Check gap with next char (if not the last char)
            if j < len(line_boxes) - 1:
                current_box = line_boxes[j]
                next_box = line_boxes[j+1]
                
                # Gap = Start of Next - End of Current
                gap = next_box['x1'] - current_box['x2']
                
                if gap > threshold:
                    line_str += " " # Add space to string
                    words_list.append(current_word) # Store finished word
                    current_word = "" # Reset
        
        # Append the last word of the line
        if current_word:
            words_list.append(current_word)

        # Store the result
        # You can choose to store the string "word1 word2" 
        # OR the list ["word1", "word2"] depending on your transformer needs
        lines_of_words.append(words_list) 

    return lines_of_words

def main():
    id_to_char = load_mapping(MAPPING_PATH)
    model = YOLO(MODEL_PATH)
    results = model.predict(IMG_PATH)
    
    # Get list of lists: [['word1', 'word2'], ['word3', 'word4']]
    structured_output = boxes_to_text_with_spaces(results, id_to_char)
    
    print("--- Extracted Words ---")
    for line in structured_output:
        print(line) 

if __name__ == "__main__":
    main()